<h1 style="text-align: center;"> 🦜 LangChain : Qdrant + Hugging Face 🤗 (Free) </h1>

![image.png](image.png)

In [90]:
#!pip install langchain

In [91]:
from langchain.vectorstores import Qdrant
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.document_loaders import TextLoader

### Hugging Face 'sentence-transformers/all-mpnet-base-v2'

In [92]:
# All-round model tuned for many use-cases. 
# Trained on a large and diverse dataset of over 1 billion training pairs.
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-mpnet-base-v2"
)

### Load text, split document(s) into chunks

In [93]:
from langchain.document_loaders import TextLoader

# 'Alice in Wonderland' ~ https://www.gutenberg.org/

loader = TextLoader(
    '/home/rag/qd/sat.txt'
)

documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

## Put into Qdrant vector database

In [94]:
qdrant = Qdrant.from_documents(
    docs, embeddings, 
    location=":memory:",  # Local mode with in-memory storage only
    collection_name="my_documents",
)

## Similarity Search using Qdrant client

In [95]:
import qdrant_client
client = qdrant_client.QdrantClient(
    "http://localhost:6333",
    api_key="", # None for local instance
)

/home/rag/.local/lib/python3.10/site-packages/qdrant_client/qdrant_remote.py:104: UserWarning: Api key is used with unsecure connection.
  warnings.warn("Api key is used with unsecure connection.")


In [96]:
query = "What color was the Rabbit?"
found_docs = qdrant.similarity_search(query)

found_docs = qdrant.similarity_search_with_score(query)
found_docs

[(Document(page_content='CHAPTER I.\nDown the Rabbit-Hole\nAlice was beginning to get very tired of sitting by her sister on the bank, and of having nothing to do: once or twice she had peeped into the book her sister was reading, but it had no pictures or conversations in it, “and what is the use of a book,” thought Alice “without pictures or conversations?”\n\nSo she was considering in her own mind (as well as she could, for the hot day made her feel very sleepy and stupid), whether the pleasure of making a daisy-chain would be worth the trouble of getting up and picking the daisies, when suddenly a White Rabbit with pink eyes ran close by her.', metadata={'source': '/home/rag/qd/sat.txt'}),
  0.5364035260609084),
 (Document(page_content='Alice was not a bit hurt, and she jumped up on to her feet in a moment: she looked up, but it was all dark overhead; before her was another long passage, and the White Rabbit was still in sight, hurrying down it. There was not a moment to be lost: a

<h1 style="text-align: center;"> 🦜 LangChain : Qdrant + OpenAI ($)</h1>

In [243]:
import os
os.environ["SERPAPI_API_KEY"] =                                                    
os.environ["OPENAI_API_KEY"] =                                                     
llm = OpenAI(temperature=0)

In [244]:
from langchain import VectorDBQA, OpenAI
from langchain.chains import RetrievalQA

In [245]:
docsearch = qdrant

In [246]:
# qa = VectorDBQA.from_chain_type(
#     llm=llm,
#     chain_type="stuff",
#     vectorstore=qdrant,
#     return_source_documents=False,
#     )

In [247]:
# `VectorDBQA` is deprecated - please use `from langchain.chains import RetrievalQA`

In [248]:
qa = RetrievalQA.from_chain_type(
    llm=OpenAI(), 
    chain_type="stuff",
    retriever=docsearch.as_retriever()
)

In [266]:
question = "What is Color was the Rabbit, exclude it's eyes?"

In [267]:
response = qa.run(question)

In [296]:
response

' White'

### Now allow user to input the question

In [297]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain.llms import OpenAI
from langchain import SerpAPIWrapper

In [298]:
tools = load_tools(["serpapi"], llm=llm)
agent = initialize_agent(tools, 
                         llm, 
                         agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, 
                         verbose=True)

In [299]:
from langchain import PromptTemplate

template = """Answer the question below in less than 4 words.
If the question cannot be answered using the information provided answer
with "I don't know". Use tool 'serpapi' if you need to find a price.

Question: {quz}

Answer: """

prompt = PromptTemplate(
    input_variables=["quz"],
    template=template
)

In [308]:
#Get the price of 5 litres gloss Wood & Metal Paint paint from wickes.co.uk
my_query = input()
my_query

Get the price of 5 litres gloss Wood & Metal Paint paint from wickes.co.uk


'Get the price of 5 litres gloss Wood & Metal Paint paint from wickes.co.uk'

In [309]:
result = agent.run(prompt.format(quz=f"{my_query} color= {response}"))



> Entering new AgentExecutor chain...
 I need to find the price of 5 litres of White gloss Wood & Metal Paint from wickes.co.uk
Action: Search
Action Input: "wickes.co.uk 5 litres White gloss Wood & Metal Paint price"
Observation: Buy great products from our white gloss wood paint Category online at Wickes.co.uk. We supply trade quality DIY and home improvement products at great low ...
Thought: I need to find the exact price
Action: Search
Action Input: "wickes.co.uk 5 litres White gloss Wood & Metal Paint price"
Observation: Buy great products from our white gloss wood paint Category online at Wickes.co.uk. We supply trade quality DIY and home improvement products at great low ...
Thought: I now know the final answer
Final Answer: £25.00

> Finished chain.
